In [1]:
#-----------------------------------------------------------------------------------
# tcm_v1.py
# Author: Emily Ryu
# test for correlation between trades in two datsaets (qty and directionality).
# v1 of the tcm (trade correlation metric) considers all trades within a window of
#       size DELTA sec following trades in the first dataset.
#-----------------------------------------------------------------------------------

import pandas as pd
import glob
import numpy as np
import datetime
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
from scipy.stats import ttest_ind
import random

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [2]:
# unpickling
filenames = glob.glob('azhang_intern_data/*.pkl')
list_of_dfs = [pd.read_pickle(f) for f in filenames]

# assign filename to each dataframe
for dataframe, filename in zip(list_of_dfs, filenames):
    start = filename.find('data/') + 5
    end = filename.find('.pkl', start)
    dataframe['filename'] = filename[start:end]

In [3]:
# create dictionary mapping filename to index
files = []
indices = []
for x in range(len(list_of_dfs)):
    files.append(list_of_dfs[x].iloc[0].filename)
    indices.append(x)
fileDict = dict(zip(files, indices))

# sortedFileDict = dict(sorted(fileDict.items()))
# for k, v in sortedFileDict.items(): print(k, v)
    
# for any date (YYYYMMDD), return a and b instrument indices as tuple (a,b)
# works for every date starting at 07.21 except 2019.08.15 bc a has aA and aB
def abIndices(date):
    for key in fileDict.keys():
        if date in key:
            if ('aA' in key) or ('aB' in key):
                aIndex = fileDict[key]
            elif ('bA' in key) or ('bB' in key) or ('bC' in key):
                bIndex = fileDict[key]
    return (aIndex, bIndex)

## calculate correlation coefficient between trades in two dfs (volume and directionality)

In [4]:
def raw_trade_corr(df1, df2, deltasec):
    # extract all trades from df1 and df2
    # we just ignore NoSide trades bc there aren't very many of them, instead of determining based on prev bundle-
    df1trades = df1.loc[(df1.msgtype == 'Trade') & ((df1.side == 'Sell') | (df1.side == 'Buy'))][['exchange_time', 'msgtype', 'qty', 'side']]
    df2trades = df2.loc[(df2.msgtype == 'Trade') & ((df2.side == 'Sell') | (df2.side == 'Buy'))][['exchange_time', 'msgtype', 'qty', 'side']]
    
    # assign - sign to sellside trades
    df1trades['sgn'] = [-1 if x == 'Sell' else 1 for x in df1trades['side']]
    df1trades['sgn_qty'] = df1trades.qty * df1trades.sgn
    df2trades['sgn'] = [-1 if x == 'Sell' else 1 for x in df2trades['side']]
    df2trades['sgn_qty'] = df2trades.qty * df2trades.sgn
    
    # combine trades that occur at the exact same timestamp
    # pandas groupby: info here https://realpython.com/pandas-groupby/, https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
    # sum both qty and sgn_qty so we can look at total volume as well as directionality
    times1 = df1trades.groupby('exchange_time').agg({'qty':sum, 'sgn_qty':sum})
    times2 = df2trades.groupby('exchange_time').agg({'qty':sum, 'sgn_qty':sum})
    
    # for each timestamp X in times1, look at X to X+delta in df2. combine trades using the same code. plot 2 vs 1 (total magnitude and signed)
    qties = []
    sgn_qties = []
    for time in times1.index:
        timeUpperBound = time + datetime.timedelta(seconds=deltasec)
        times2range = times2.loc[(times2.index >= time) & (times2.index <= timeUpperBound)]
        qties.append((times1.loc[time].get('qty'), times2range.sum(0).get('qty')))
        sgn_qties.append((times1.loc[time].get('sgn_qty'), times2range.sum(0).get('sgn_qty')))
    
    # calculate correlation
    x_qt = [x[0] for x in qties]
    y_qt = [x[1] for x in qties]
    x_sqt = [x[0] for x in sgn_qties]
    y_sqt = [x[1] for x in sgn_qties]
    
    # if y_qt or y_sqt is all zero, return 0 for that part of the correlation
    if all(v == 0 for v in y_qt):
        if all(w == 0 for w in y_sqt):
            return (0,0)
        return (0, pearsonr(x_sqt, y_sqt)[0])
    if all(w == 0 for w in y_sqt):
        return (pearsonr(x_qt, y_qt)[0], 0)
    return (pearsonr(x_qt, y_qt)[0], pearsonr(x_sqt, y_sqt)[0])

## correlation metrics -- for a single day, or for any two dfs

In [5]:
# correlation metric for a single day, scaled by the two dfs compared with themselves
def day_correlation(date, delta):   # note: date should be a string YYYYMMDD, delta is an int or float (seconds)
    indices = abIndices(date)
    dfA = list_of_dfs[indices[0]]
    dfB = list_of_dfs[indices[1]]
    unscaled = raw_trade_corr(dfA, dfB, delta)
    scaleFactor0 = max(raw_trade_corr(dfA, dfA, delta)[0], raw_trade_corr(dfB, dfB, DELTA)[0])
    scaleFactor1 = max(raw_trade_corr(dfA, dfA, delta)[1], raw_trade_corr(dfB, dfB, DELTA)[1])
    return (unscaled[0] / scaleFactor0, unscaled[1] / scaleFactor1)

# correlation metric testing for two dataframes, scaled by the two dfs compared with themselves
def df_correlation(df1, df2, delta):   # note: date should be a string YYYYMMDD, delta is an int or float (seconds)
    unscaled = raw_trade_corr(df1, df2, delta)
    scaleFactor0 = max(abs(raw_trade_corr(df1, df1, delta)[0]), abs(raw_trade_corr(df2, df2, delta)[0]))
    scaleFactor1 = max(abs(raw_trade_corr(df1, df1, delta)[1]), abs(raw_trade_corr(df2, df2, delta)[1]))
    return (unscaled[0] / scaleFactor0, unscaled[1] / scaleFactor1)

In [17]:
# testing
DELTA = 150
df = list_of_dfs[abIndices('20191003')[1]]
df.loc[df.msgtype == 'Trade'][['exchange_time', 'msgtype', 'qty', 'side']]
print(day_correlation('20190722', DELTA))
print(df_correlation(df, df, DELTA))  # should always be (1,1) when compared with self

(0.7557760649096326, 0.48934897798600546)
(1.0, 1.0)


## a and b are related! (much more than random datasets)

In [18]:
# run day_correlation for every set of data we have
# extract unique dates where we have data for both a and b
dates = []
for filename in fileDict.keys():
    if 'bA' in filename or 'bB' in filename or 'bC' in filename:
        if '20190815' not in filename:
            dates.append(int(filename[3:]))      # represent as ints so that we can sort chronologically (something funky happens w strings)
dates = sorted(dates)

sameDateQties = []
sameDateSqties = []
for dateint in dates:
    dc = day_correlation(str(dateint), DELTA)
    sameDateQties.append(dc[0])
    sameDateSqties.append(dc[1])

In [19]:
print(sameDateQties)
print(sameDateSqties)
print(np.mean(sameDateQties))
print(np.mean(sameDateSqties))

[0.7670740659165678, 0.7557760649096326, 0.663849684662856, 0.8235482440387778, 0.9941677932365808, 0.9083578366320779, 0.5670468078855363, 1.005478532161837, 0.36916779231287367, 0.7451819767713675, 0.806745967698566, 0.7028201570192756, 0.756575030421825, 0.6126218261336156, 0.6412991520129032, 0.4013556212922955, 0.3005753810268873, 0.6912114813783885, 0.7995960450895354, 0.5088647387957074, 0.7789022560344839, 0.7904716975159921, 0.3406030315218723, 0.8830131368730827, 0.7862998621962356, 0.5352607017376039, 0.6529747543155631, 0.24615194443258454, 0.5959062661934346, 0.6440287720696483, 0.5396547364361199, 0.2529679359040096, 0.34296253775858687, 0.19874140270648422, 0.5352594323333716, 0.3777389821662374, 0.6035272801296587, 0.3978761065729971, 0.2802806706400257, 0.42259487944069923, 0.3709970967983309, 0.28799024483267716, 0.2002569322964414, 0.6040323261291987, 0.41862223237324364, 0.3406380293537845, 0.567771381198451, 0.5851362396193788, 0.5004413340743956, 0.387588990101206

In [20]:
# run df_correlation for a set of pairs of random dfs
randomTuples = []
randomQties = []
randomSqties = []

for i in range(len(dates)):
    index1 = random.randrange(len(list_of_dfs))
    index2 = random.randrange(len(list_of_dfs))
    while (index2 == index1):
        index2 = random.randrange(len(list_of_dfs)) # make sure we're not comparing with self
    randomTuples.append((index1,index2))
    dfc = df_correlation(list_of_dfs[index1], list_of_dfs[index2], DELTA)
    randomQties.append(dfc[0])
    randomSqties.append(dfc[1])

In [21]:
print(randomTuples)
print(randomQties)
print(randomSqties)

[(43, 124), (36, 110), (50, 112), (39, 31), (54, 36), (94, 3), (29, 68), (125, 14), (51, 68), (75, 106), (74, 34), (19, 42), (20, 68), (108, 14), (55, 109), (107, 54), (23, 27), (48, 126), (17, 68), (101, 62), (109, 107), (96, 107), (128, 21), (38, 101), (23, 103), (62, 17), (71, 9), (28, 78), (20, 87), (62, 16), (123, 10), (117, 44), (52, 67), (100, 86), (121, 113), (32, 104), (83, 19), (55, 95), (52, 120), (83, 15), (18, 43), (39, 46), (3, 97), (67, 127), (99, 78), (68, 49), (89, 115), (93, 66), (110, 119), (12, 83), (0, 112), (42, 47), (24, 20), (0, 47), (87, 14), (103, 119), (109, 75), (104, 99), (38, 59), (1, 83), (57, 40)]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [22]:
# t test for statistcal significance difference bt the two
print(ttest_ind(sameDateQties, randomQties, equal_var = False))
print(ttest_ind(sameDateSqties, randomSqties, equal_var = False))

Ttest_indResult(statistic=18.45340340265046, pvalue=2.044640621503051e-26)
Ttest_indResult(statistic=10.197043615930488, pvalue=1.015910848302365e-14)


## a leading b has greater correlation than b leading a

In [23]:
def day_correlation_BA(date, delta):   # note: date should be a string YYYYMMDD, delta is an int (seconds)
    indices = abIndices(date)
    dfA = list_of_dfs[indices[0]]
    dfB = list_of_dfs[indices[1]]
    unscaled = raw_trade_corr(dfB, dfA, delta)   # switch order of b and a code otherwise same
    scaleFactor0 = max(raw_trade_corr(dfA, dfA, delta)[0], raw_trade_corr(dfB, dfB, DELTA)[0])
    scaleFactor1 = max(raw_trade_corr(dfA, dfA, delta)[1], raw_trade_corr(dfB, dfB, DELTA)[1])
    return (unscaled[0] / scaleFactor0, unscaled[1] / scaleFactor1)

In [25]:
sameDateQtiesBA = []
sameDateSqtiesBA = []
for dateint in dates:
    dc = day_correlation_BA(str(dateint), DELTA)
    sameDateQtiesBA.append(dc[0])
    sameDateSqtiesBA.append(dc[1])
print(sameDateQtiesBA)
print(sameDateSqtiesBA)

[0.5837122980400278, 0.5453005387361219, 0.2887315282030423, 0.6241472313424755, 0.4956321080084221, 0.2646199454942585, -0.17913191624685548, 0.25421261765007575, 0.1949917512265915, 0.6324796694208624, 0.7131090319646339, 0.8082460804592848, 0.7310364186952805, 0.15222065483099112, 0.3265363892071796, 0.39373895210148113, 0.493417350900276, 0.33873150990085366, 0.5439157451975081, 0.4711865958466211, 0.1823467145835507, 0.3354516512828031, 0.2821043393579923, 0.6939543948251553, 0.3229953262942533, 0.46664944761790755, 0.2523545086531893, 0.10406619811173887, 0.8895898305493074, 0.6346422094339784, 0.5476863513662183, 0.7584993325551227, 0.8134915717113833, 0.1239928451516413, 0.48451695013504525, 0.4571501338256851, 0.08652357833477002, 0.5756421593995734, 0.596702599098158, 0.22597811578881719, 0.19820786254712622, 0.4268810941865628, 0.04705425638449214, 0.4462576882129789, 0.16110888388522474, 0.41016831679011456, 1.0039421929835664, 0.44885753626194785, 0.6786378323255333, 0.618

In [26]:
print(np.mean(sameDateQtiesBA))
print(np.mean(sameDateSqtiesBA))
print(ttest_ind(sameDateQties, sameDateQtiesBA, equal_var = False))
print(ttest_ind(sameDateSqties, sameDateSqtiesBA, equal_var = False))

0.43226643165657325
0.20521361545065595
Ttest_indResult(statistic=2.4549019929931633, pvalue=0.015533142510015007)
Ttest_indResult(statistic=-0.6737064389185259, pvalue=0.5018336757513682)
